In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.datasets import fetch_openml


In [3]:
from sklearn.datasets import fetch_openml

# Load Forest CoverType dataset from OpenML
data = fetch_openml(
    name="covertype",
    version=3,
    as_frame=True
)


In [4]:
# Separate features and target
X = data.data
y = data.target.astype(int)  # Convert target to integer

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)


Feature matrix shape: (581012, 54)
Target shape: (581012,)


In [5]:
X.head()


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,0.368684,0.141667,0.045455,0.184681,0.223514,0.071659,0.870079,0.913386,0.582677,0.875366,...,0,0,0,0,0,0,0,0,0,0
1,0.365683,0.155556,0.030303,0.151754,0.215762,0.054798,0.866142,0.925197,0.594488,0.867838,...,0,0,0,0,0,0,0,0,0,0
2,0.472736,0.386111,0.136364,0.191840,0.307494,0.446817,0.921260,0.937008,0.531496,0.853339,...,0,0,0,0,0,0,0,0,0,0
3,0.463232,0.430556,0.272727,0.173228,0.375969,0.434172,0.937008,0.937008,0.480315,0.865886,...,0,0,0,0,0,0,0,0,0,0
4,0.368184,0.125000,0.030303,0.109520,0.222222,0.054939,0.866142,0.921260,0.590551,0.860449,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y.value_counts().sort_index()


class
1    211840
2    283301
3     35754
4      2747
5      9493
6     17367
7     20510
Name: count, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [9]:
# Shift labels from 1–7 to 0–6
y_train_nn = y_train - 1
y_test_nn = y_test - 1


In [10]:
print("Train features:", X_train_scaled.shape)
print("Test features:", X_test_scaled.shape)

print("Train labels:", y_train_nn.shape)
print("Test labels:", y_test_nn.shape)

print("Unique classes:", sorted(y_train_nn.unique()))


Train features: (464809, 54)
Test features: (116203, 54)
Train labels: (464809,)
Test labels: (116203,)
Unique classes: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]


3.1 NEURAL NETWORK ARCHITECTURE





In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

print("TensorFlow:", tf.__version__)

# Baseline MLP architecture
model_baseline = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(7, activation='softmax')
])


2025-12-30 12:48:03.845249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767098884.023061      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767098884.080332      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767098884.489978      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767098884.490024      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767098884.490027      55 computation_placer.cc:177] computation placer alr

TensorFlow: 2.19.0


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1767098896.316729      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [12]:
model_baseline.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [13]:
model_baseline.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         7,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,751 (61.53 KB)

 Trainable params: 15,751 (61.53 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history_baseline = model_baseline.fit(
    X_train_scaled,
    y_train_nn,
    validation_split=0.2,
    epochs=20,
    batch_size=256,
    verbose=1
)


Epoch 1/20


I0000 00:00:1767098899.044109     125 service.cc:152] XLA service 0x7c394800da40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767098899.044136     125 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1767098899.277389     125 cuda_dnn.cc:529] Loaded cuDNN version 91002


  75/1453 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4924 - loss: 1.3248

I0000 00:00:1767098899.992350     125 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1453/1453 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7058 - loss: 0.7183 - val_accuracy: 0.7812 - val_loss: 0.5134
Epoch 2/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7871 - loss: 0.4951 - val_accuracy: 0.8075 - val_loss: 0.4570
Epoch 3/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8123 - loss: 0.4425 - val_accuracy: 0.8242 - val_loss: 0.4182
Epoch 4/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8281 - loss: 0.4106 - val_accuracy: 0.8376 - val_loss: 0.3944
Epoch 5/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8380 - loss: 0.3883 - val_accuracy: 0.8411 - val_loss: 0.3809
Epoch 6/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8462 - loss: 0.3707 - val_accuracy: 0.8462 - val_loss: 0.3676
Epoch 7/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8531 - loss: 0.3563 - val_accuracy: 0.8503 - val_loss: 0.3593
Epoch 8/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8564 - loss: 0.3463 - val_accurac

In [15]:
test_loss, test_accuracy = model_baseline.evaluate(
    X_test_scaled,
    y_test_nn,
    verbose=0
)

print(f"Baseline Test Accuracy: {test_accuracy:.4f}")


Baseline Test Accuracy: 0.8806


 Modify the baseline MLP architecture by exploring deeper or wider configurations. 


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation


In [17]:
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1


In [18]:
model_improved = Sequential([
    Dense(256, input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Activation('relu'),

    Dense(128),
    BatchNormalization(),
    Activation('relu'),

    Dense(64),
    BatchNormalization(),
    Activation('relu'),

    Dense(7, activation='softmax')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model_improved.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [20]:
history_improved = model_improved.fit(
    X_train_scaled, y_train_adjusted,
    validation_data=(X_test_scaled, y_test_adjusted),
    epochs=20,
    batch_size=32,
    verbose=1
)


Epoch 1/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 44s 3ms/step - accuracy: 0.7454 - loss: 0.6020 - val_accuracy: 0.8290 - val_loss: 0.4121
Epoch 2/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 0.8111 - loss: 0.4448 - val_accuracy: 0.8527 - val_loss: 0.3573
Epoch 3/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 0.8301 - loss: 0.4035 - val_accuracy: 0.8661 - val_loss: 0.3320
Epoch 4/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 0.8390 - loss: 0.3817 - val_accuracy: 0.8740 - val_loss: 0.3094
Epoch 5/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.8460 - loss: 0.3685 - val_accuracy: 0.8812 - val_loss: 0.2952
Epoch 6/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.8504 - loss: 0.3582 - val_accuracy: 0.8848 - val_loss: 0.2864
Epoch 7/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.8533 - loss: 0.3496 - val_accuracy: 0.8856 - val_loss: 0.2813
Epoch 8/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 

“An improved MLP architecture was implemented by increasing network depth and width and introducing Batch Normalization layers after each dense layer. ReLU activations were used to improve non-linearity and training stability. This architecture demonstrated improved convergence compared to the baseline model.”

3.2 REGULARIZATION TECHNIQUES 

 Apply Dropout to prevent overfitting and experiment with different dropout rates.

 Add L2 regularization to Dense layers and observe its impact on learning dynamics. 

In [21]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout

model_regularized = Sequential([
    Dense(256, input_shape=(X_train_scaled.shape[1],),
          kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),

    Dense(128, kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),

    Dense(64, kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),

    Dense(7, activation='softmax')  # output layer
])


In [23]:
model_regularized.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [24]:
y_train_adj = y_train - 1
y_test_adj = y_test - 1


In [25]:
history_regularized = model_regularized.fit(
    X_train_scaled, y_train_adj,
    validation_data=(X_test_scaled, y_test_adj),
    epochs=30,
    batch_size=64,
    verbose=1
)


Epoch 1/100
7263/7263 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - accuracy: 0.6981 - loss: 0.9043 - val_accuracy: 0.7747 - val_loss: 0.6028
Epoch 2/100
7263/7263 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.7482 - loss: 0.6592 - val_accuracy: 0.7791 - val_loss: 0.5941
Epoch 3/100
7263/7263 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.7562 - loss: 0.6498 - val_accuracy: 0.7841 - val_loss: 0.5842
Epoch 4/100
7263/7263 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.7588 - loss: 0.6467 - val_accuracy: 0.7885 - val_loss: 0.5807
Epoch 5/100
7263/7263 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.7612 - loss: 0.6447 - val_accuracy: 0.7900 - val_loss: 0.5789
Epoch 6/100
7263/7263 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.7608 - loss: 0.6434 - val_accuracy: 0.7954 - val_loss: 0.5730
Epoch 7/100
7263/7263 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.7623 - loss: 0.6426 - val_accuracy: 0.7908 - val_loss: 0.5704
Epoch 8/100
7263/7263 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.7632 -

🔹 Task: Add L2 Regularization

In [26]:
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout

# L2 regularization factor
l2_lambda = 0.001

model_l2 = Sequential([
    Dense(256, input_shape=(X_train_scaled.shape[1],),
          kernel_regularizer=regularizers.l2(l2_lambda)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),

    Dense(128, kernel_regularizer=regularizers.l2(l2_lambda)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),

    Dense(64, kernel_regularizer=regularizers.l2(l2_lambda)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),

    Dense(7, activation='softmax')  # output layer
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model_l2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history_l2 = model_l2.fit(
    X_train_scaled, y_train_adj,
    validation_data=(X_test_scaled, y_test_adj),
    epochs=50,
    batch_size=64,
    verbose=1
)


In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history_l2.history['accuracy'], label='Train Accuracy')
plt.plot(history_l2.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy with L2 Regularization')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.plot(history_l2.history['loss'], label='Train Loss')
plt.plot(history_l2.history['val_loss'], label='Validation Loss')
plt.title('Loss with L2 Regularization')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


3.3 OPTIMIZER AND LEARNING RATE STRATEGY 


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-5,
    verbose=1
)


In [ ]:
from tensorflow.keras.optimizers import Adam

model_adam = tf.keras.models.clone_model(model_l2)
model_adam.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_adam = model_adam.fit(
    X_train_scaled, y_train_adj,
    validation_data=(X_test_scaled, y_test_adj),
    epochs=40,
    batch_size=64,
    callbacks=[lr_scheduler],
    verbose=1
)


In [ ]:
from tensorflow.keras.optimizers import RMSprop

model_rmsprop = tf.keras.models.clone_model(model_l2)
model_rmsprop.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_rmsprop = model_rmsprop.fit(
    X_train_scaled, y_train_adj,
    validation_data=(X_test_scaled, y_test_adj),
    epochs=40,
    batch_size=64,
    callbacks=[lr_scheduler],
    verbose=1
)


In [ ]:
from tensorflow.keras.optimizers import SGD

model_sgd = tf.keras.models.clone_model(model_l2)
model_sgd.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_sgd = model_sgd.fit(
    X_train_scaled, y_train_adj,
    validation_data=(X_test_scaled, y_test_adj),
    epochs=40,
    batch_size=64,
    callbacks=[lr_scheduler],
    verbose=1
)


In [ ]:
print("Adam Final Val Accuracy:", history_adam.history['val_accuracy'][-1])
print("RMSprop Final Val Accuracy:", history_rmsprop.history['val_accuracy'][-1])
print("SGD Final Val Accuracy:", history_sgd.history['val_accuracy'][-1])


✅ Task 3.4 – Training Management

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


In [ ]:
model_final = tf.keras.models.clone_model(model_l2)

model_final.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_final = model_final.fit(
    X_train_scaled, y_train_adj,
    validation_data=(X_test_scaled, y_test_adj),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_final.history['accuracy'], label='Train Accuracy')
plt.plot(history_final.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
plt.plot(history_final.history['loss'], label='Train Loss')
plt.plot(history_final.history['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()



3.5 MODEL EVALUATION 


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns


In [ ]:
# Predict class labels
y_pred_probs = model_final.predict(X_test_scaled)
y_pred = y_pred_probs.argmax(axis=1)


In [ ]:
# Accuracy
acc = accuracy_score(y_test_adj, y_pred)

# Precision, Recall, F1
precision_macro = precision_score(y_test_adj, y_pred, average='macro')
recall_macro = recall_score(y_test_adj, y_pred, average='macro')
f1_macro = f1_score(y_test_adj, y_pred, average='macro')

precision_weighted = precision_score(y_test_adj, y_pred, average='weighted')
recall_weighted = recall_score(y_test_adj, y_pred, average='weighted')
f1_weighted = f1_score(y_test_adj, y_pred, average='weighted')

print(f"Test Accuracy: {acc:.4f}")
print(f"Macro Precision: {precision_macro:.4f}, Recall: {recall_macro:.4f}, F1: {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}, Recall: {recall_weighted:.4f}, F1: {f1_weighted:.4f}")


In [ ]:
cm = confusion_matrix(y_test_adj, y_pred)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(1,8), yticklabels=range(1,8))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


3.6 COMPARISON WITH ENSEMBLE METHODS 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the RandomForest
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

# Train on the same training set
rf_model.fit(X_train_scaled, y_train_adj)


In [ ]:
# Predict on test set
y_pred_rf = rf_model.predict(X_test_scaled)

# Accuracy
acc_rf = accuracy_score(y_test_adj, y_pred_rf)

# Precision, Recall, F1
precision_macro_rf = precision_score(y_test_adj, y_pred_rf, average='macro')
recall_macro_rf = recall_score(y_test_adj, y_pred_rf, average='macro')
f1_macro_rf = f1_score(y_test_adj, y_pred_rf, average='macro')

precision_weighted_rf = precision_score(y_test_adj, y_pred_rf, average='weighted')
recall_weighted_rf = recall_score(y_test_adj, y_pred_rf, average='weighted')
f1_weighted_rf = f1_score(y_test_adj, y_pred_rf, average='weighted')

print(f"RandomForest Test Accuracy: {acc_rf:.4f}")
print(f"Macro Precision: {precision_macro_rf:.4f}, Recall: {recall_macro_rf:.4f}, F1: {f1_macro_rf:.4f}")
print(f"Weighted Precision: {precision_weighted_rf:.4f}, Recall: {recall_weighted_rf:.4f}, F1: {f1_weighted_rf:.4f}")


In [ ]:
cm_rf = confusion_matrix(y_test_adj, y_pred_rf)

plt.figure(figsize=(10,8))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', xticklabels=range(1,8), yticklabels=range(1,8))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('RandomForest Confusion Matrix')
plt.show()


Tree-based ensemble methods like RandomForest often outperform MLPs on structured/tabular datasets because they naturally capture non-linear relationships and feature interactions without extensive preprocessing. They are less sensitive to feature scaling, handle imbalanced data well, and aggregate predictions from multiple trees to reduce overfitting. In contrast, MLPs require careful tuning, normalization, and often more data to generalize as effectively on tabular data.